First add the data functions to the path

In [1]:
import sys
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from pathlib import Path
notebook_dir = !pwd # Replace with !echo %cd% if using windows
library_dir = Path(notebook_dir[0]).parent.absolute()
sys.path.append(str(library_dir))
data_raw_path = library_dir / 'data' / 'raw'

from src.data.adcp_mission_analysis import adcp_import_data

Use better plotting defaults

In [19]:
import matplotlib.pyplot as plt
from matplotlib import style
style_path = library_dir / 'src' / 'visualization' / 'presentation.mplstyle'
style.use(str(style_path))

Get an overview of the dataset

In [20]:
overview, mission_dict = adcp_import_data(data_raw_path)

In [10]:
overview.head()

,file_path,dive_limb,powerusage_mW,mem_usage_MB_per_hour,cell_size,measurement_interval,num_cells,num_pings,blank_dist,vert_direction
0003a,/media/callum/storage/Documents/adcp-glider/da...,a,184.039993,0.061,2.0,15,15,4,0.3,Descent
0003b,/media/callum/storage/Documents/adcp-glider/da...,b,184.039993,0.061,2.0,15,15,4,0.3,Ascent
0004a,/media/callum/storage/Documents/adcp-glider/da...,a,184.039993,0.061,2.0,15,15,4,0.3,Descent
0004b,/media/callum/storage/Documents/adcp-glider/da...,b,184.039993,0.061,2.0,15,15,4,0.3,Ascent
0005a,/media/callum/storage/Documents/adcp-glider/da...,a,184.039993,0.061,2.0,15,15,4,0.3,Descent


In [22]:
overview['measurement_interval']



0003a    15
0003b    15
0004a    15
0004b    15
0005a    15
0005b    15
0008b    15
0009a    15
0009b    15
0010a    15
0010b    15
0011a    15
0011b    15
0012a    15
Name: measurement_interval, dtype: int64